In [1]:
import pandas as pd
import os
from tqdm.notebook import tqdm

class SMT_test:
    chart = None
    def __init__(self, benchmarks_path='../../', chart_path='../charts', result_path='../results'):
        self.benchmarks_path = benchmarks_path
        self.chart_path = chart_path
        self.result_path = result_path

    def init_chart(self, folder_path, read_path, save_path):
        full_execl_path = f'{self.chart_path}/{read_path}'
        self.chart = pd.read_excel(full_execl_path, header=[0], index_col = [0])
        full_folder_path = f'{self.benchmarks_path}{folder_path}'
        self.get_info(full_folder_path)
        full_save_path = f'{self.chart_path}/{save_path}'
        self.chart.to_excel(full_save_path)
    
    def get_info(self, folder_path):
        benchmarks = os.listdir(folder_path)
        for file in benchmarks:
            path = os.path.join(folder_path, file)
            if os.path.isdir(path):
                self.get_info(path)
            else:
                try:
                    with open(path) as f:
                        vars = 0
                        assertions = 0
                        status = 'none'
                        line = f.readline()
                        while line:
                            if line[:3] == '(de':
                                vars += 1
                            elif line[:3] == '(as':
                                assertions += 1
                            elif line[11:17] == 'status':
                                status = line.split(' ')[2][:-2]
                            line = f.readline()
                        category = path.split('/')[-2]
                        path = path.replace(self.benchmarks_path, '')
                        self.chart.loc[len(self.chart.index)] = [path, category, vars, assertions, status]
                except:
                    print(path)

    def arrange(self, solver, log_path, read_path, save_path):
        full_execl_path = f'{self.chart_path}/{read_path}'
        self.chart = pd.read_excel(full_execl_path, header=[0], index_col = [0])
        full_res_path = f'{self.result_path}/{solver}/{log_path}'
        results = os.listdir(full_res_path)
        result_name = f'result_{solver}_{log_path}'
        time_name = f'time_{solver}_{log_path}'
        total = len(self.chart.index)
        count = 0
        for file in tqdm(results, desc=full_res_path):
            with open(os.path.join(full_res_path, file)) as f:
                try:
                    if solver == 'cvc5':
                        while f.readline():
                            info = f.readline().rstrip()
                            result = info
                            f.readline()
                            if info == 'unknown':
                                info = f.readline().rstrip()
                                if len(info) < 10:
                                    if len(info) > 1:
                                        result = info
                                    info = f.readline().rstrip()
                            elif len(info) > 10:
                                result = "unknown"
                            else:
                                info = f.readline().rstrip()
                            name = info.split(' : ')[0]
                            timeout = info.split(' : ')[1][:-4]
                            self.chart.loc[self.chart['filename'] == name, [result_name, time_name]] = [result, float(timeout)]
                            count += 1
                    elif solver == 'cvc5_new':
                        while f.readline():
                            info = f.readline().rstrip()
                            result = info
                            if len(info) > 10:
                                result = "unknown"
                            info = f.readline().rstrip()
                            name = info.split(' : ')[0]
                            timeout = info.split(' : ')[1][:-4]
                            self.chart.loc[self.chart['filename'] == name, [result_name, time_name]] = [result, float(timeout)]
                            count += 1
                    elif solver == 'z3':
                        while f.readline():
                            info = f.readline().rstrip()
                            result = info
                            info = f.readline().rstrip()
                            name = info.split(' : ')[0]
                            timeout = info.split(' : ')[1][:-4]
                            self.chart.loc[self.chart['filename'] == name, [result_name, time_name]] = [result, float(timeout)]
                            count += 1
                except:
                    print(file)

        print(f"count/total: {count}/{total}")
        
        full_save_path = f'{self.chart_path}/{save_path}'
        self.chart.to_excel(full_save_path)

In [3]:
str = SMT_test()
str.arrange('cvc5', 'QF_SLIA', 'QF_SLIA.xlsx', 'result_QF_SLIA_cvc5.xlsx')
str.arrange('z3', 'QF_SLIA', 'QF_SLIA.xlsx', 'result_QF_SLIA_z3.xlsx')

../results/cvc5/QF_SLIA:   0%|          | 0/39 [00:00<?, ?it/s]

count/total: 427/81109


../results/z3/QF_SLIA:   0%|          | 0/39 [00:00<?, ?it/s]

count/total: 428/81109
